In [1]:
import pandas as pd
import numpy as np
import csv
import os

In [13]:


# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open("C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_taggedartists-timestamps.txt").readlines()]
df=pd.DataFrame(datContent[1:],columns=datContent[0])
df.to_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_taggedartists-timestamps.csv")

In [11]:
df=pd.DataFrame(datContent[1:],columns=datContent[0])

In [47]:
# 遍历文件夹
def walkFile(file):
    filelist=[]
    for root, dirs, files in os.walk(file):

        # root 表示当前正在访问的文件夹路径
        # dirs 表示该文件夹下的子目录名list
        # files 表示该文件夹下的文件list

        # 遍历文件
        for f in files:
            filelist.append(os.path.join(f))

        return filelist
filelist=walkFile("C:/Users/19223/Downloads/mab/experi/data/0lastfm")
filelist

['artists.txt',
 'readme.txt',
 'tags.txt',
 'user_artists.txt',
 'user_friends.txt',
 'user_taggedartists-timestamps.txt',
 'user_taggedartists.txt']

In [599]:
#convert to csv
datContent = [i.strip().split("	") for i in open("C:/Users/19223/Downloads/mab/experi/data/0lastfm/artists.txt",'r', encoding='UTF-8').readlines()]
df=pd.DataFrame(datContent[1:],columns=datContent[0])
df.to_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/artists.csv")
print("C:/Users/19223/Downloads/mab/experi/data/0lastfm/artists.csv")
for i in filelist[2:]:
    datContent = [i.strip().split("	") for i in open("C:/Users/19223/Downloads/mab/experi/data/0lastfm/"+i).readlines()]
    df=pd.DataFrame(datContent[1:],columns=datContent[0])
    i=i.strip(".txt")+".csv"
    df.to_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/"+i)
    print("C:/Users/19223/Downloads/mab/experi/data/0lastfm/"+i)

C:/Users/19223/Downloads/mab/experi/data/0lastfm/artists.csv
C:/Users/19223/Downloads/mab/experi/data/0lastfm/ags.csv
C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_artists.csv
C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_friends.csv
C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_taggedartists-timestamps.csv
C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_taggedartists.csv


In [629]:
artists=pd.read_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/artists.csv")
artists.drop(['Unnamed: 0'],axis=1, inplace=True)
tags=pd.read_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/ags.csv")
tags.drop(['Unnamed: 0'],axis=1, inplace=True)
user_artists=pd.read_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_artists.csv")
user_artists.drop(['Unnamed: 0'],axis=1, inplace=True)
user_friends=pd.read_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_friends.csv")
user_friends.drop(['Unnamed: 0'],axis=1, inplace=True)
user_taggedartists_timestamps=pd.read_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_taggedartists-timestamps.csv")
user_taggedartists_timestamps.drop(['Unnamed: 0'],axis=1, inplace=True)
user_taggedartists_timestamps=user_taggedartists_timestamps.sort_values(by=['timestamp'])
user_taggedartists=pd.read_csv("C:/Users/19223/Downloads/mab/experi/data/0lastfm/user_taggedartists.csv")
user_taggedartists.drop(['Unnamed: 0'],axis=1, inplace=True)

In [630]:
#100 most frequent users
frequentUser=np.array(user_taggedartists.userID.value_counts().index[:60])
#10 most frequent items
frequentItem=list(user_taggedartists.artistID.value_counts().index[:300])
frequentUserData=user_taggedartists_timestamps[(user_taggedartists_timestamps['userID'].isin(frequentUser))].sort_values(by=['timestamp'])
freqData=user_taggedartists_timestamps[(user_taggedartists_timestamps['userID'].isin(frequentUser))&(user_taggedartists_timestamps['artistID'].isin(frequentItem))].sort_values(by=['timestamp'])
userFlow=list(freqData.userID)
trueResultFlow=list(freqData.artistID)
len(freqData)

15301

In [602]:
import jieba  
import jieba.posseg as pseg  
import os  
import sys  
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

corpus=list(tags.tagValue)
vectorizer=CountVectorizer()#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频  
transformer=TfidfTransformer()#该类会统计每个词语的tf-idf权值  
tfidf=transformer.fit_transform(vectorizer.fit_transform(corpus))#第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵  
word=vectorizer.get_feature_names()#获取词袋模型中的所有词语  
weight=tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重  
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重  
    print( u"-------这里输出第",i,u"类文本的词语tf-idf权重------"  )
print( 'Start Kmeans:')

clf = KMeans(n_clusters=4)   #科技 医学 汽车 国家
s = clf.fit(weight)
#每个样本所属的簇
label = []              
#print(clf.labels_)
i = 1
while i <= len(clf.labels_):
    #print( clf.labels_[i-1])
    label.append(clf.labels_[i-1])
    i = i + 1


pca = PCA(n_components=10)
newData = pca.fit_transform(weight)   #载入N维

-------这里输出第 0 类文本的词语tf-idf权重------
-------这里输出第 1 类文本的词语tf-idf权重------
-------这里输出第 2 类文本的词语tf-idf权重------
-------这里输出第 3 类文本的词语tf-idf权重------
-------这里输出第 4 类文本的词语tf-idf权重------
-------这里输出第 5 类文本的词语tf-idf权重------
-------这里输出第 6 类文本的词语tf-idf权重------
-------这里输出第 7 类文本的词语tf-idf权重------
-------这里输出第 8 类文本的词语tf-idf权重------
-------这里输出第 9 类文本的词语tf-idf权重------
-------这里输出第 10 类文本的词语tf-idf权重------
-------这里输出第 11 类文本的词语tf-idf权重------
-------这里输出第 12 类文本的词语tf-idf权重------
-------这里输出第 13 类文本的词语tf-idf权重------
-------这里输出第 14 类文本的词语tf-idf权重------
-------这里输出第 15 类文本的词语tf-idf权重------
-------这里输出第 16 类文本的词语tf-idf权重------
-------这里输出第 17 类文本的词语tf-idf权重------
-------这里输出第 18 类文本的词语tf-idf权重------
-------这里输出第 19 类文本的词语tf-idf权重------
-------这里输出第 20 类文本的词语tf-idf权重------
-------这里输出第 21 类文本的词语tf-idf权重------
-------这里输出第 22 类文本的词语tf-idf权重------
-------这里输出第 23 类文本的词语tf-idf权重------
-------这里输出第 24 类文本的词语tf-idf权重------
-------这里输出第 25 类文本的词语tf-idf权重------
-------这里输出第 26 类文本的词语tf-idf权重------
-------这里输出

-------这里输出第 718 类文本的词语tf-idf权重------
-------这里输出第 719 类文本的词语tf-idf权重------
-------这里输出第 720 类文本的词语tf-idf权重------
-------这里输出第 721 类文本的词语tf-idf权重------
-------这里输出第 722 类文本的词语tf-idf权重------
-------这里输出第 723 类文本的词语tf-idf权重------
-------这里输出第 724 类文本的词语tf-idf权重------
-------这里输出第 725 类文本的词语tf-idf权重------
-------这里输出第 726 类文本的词语tf-idf权重------
-------这里输出第 727 类文本的词语tf-idf权重------
-------这里输出第 728 类文本的词语tf-idf权重------
-------这里输出第 729 类文本的词语tf-idf权重------
-------这里输出第 730 类文本的词语tf-idf权重------
-------这里输出第 731 类文本的词语tf-idf权重------
-------这里输出第 732 类文本的词语tf-idf权重------
-------这里输出第 733 类文本的词语tf-idf权重------
-------这里输出第 734 类文本的词语tf-idf权重------
-------这里输出第 735 类文本的词语tf-idf权重------
-------这里输出第 736 类文本的词语tf-idf权重------
-------这里输出第 737 类文本的词语tf-idf权重------
-------这里输出第 738 类文本的词语tf-idf权重------
-------这里输出第 739 类文本的词语tf-idf权重------
-------这里输出第 740 类文本的词语tf-idf权重------
-------这里输出第 741 类文本的词语tf-idf权重------
-------这里输出第 742 类文本的词语tf-idf权重------
-------这里输出第 743 类文本的词语tf-idf权重------
-------这里输出第

-------这里输出第 1593 类文本的词语tf-idf权重------
-------这里输出第 1594 类文本的词语tf-idf权重------
-------这里输出第 1595 类文本的词语tf-idf权重------
-------这里输出第 1596 类文本的词语tf-idf权重------
-------这里输出第 1597 类文本的词语tf-idf权重------
-------这里输出第 1598 类文本的词语tf-idf权重------
-------这里输出第 1599 类文本的词语tf-idf权重------
-------这里输出第 1600 类文本的词语tf-idf权重------
-------这里输出第 1601 类文本的词语tf-idf权重------
-------这里输出第 1602 类文本的词语tf-idf权重------
-------这里输出第 1603 类文本的词语tf-idf权重------
-------这里输出第 1604 类文本的词语tf-idf权重------
-------这里输出第 1605 类文本的词语tf-idf权重------
-------这里输出第 1606 类文本的词语tf-idf权重------
-------这里输出第 1607 类文本的词语tf-idf权重------
-------这里输出第 1608 类文本的词语tf-idf权重------
-------这里输出第 1609 类文本的词语tf-idf权重------
-------这里输出第 1610 类文本的词语tf-idf权重------
-------这里输出第 1611 类文本的词语tf-idf权重------
-------这里输出第 1612 类文本的词语tf-idf权重------
-------这里输出第 1613 类文本的词语tf-idf权重------
-------这里输出第 1614 类文本的词语tf-idf权重------
-------这里输出第 1615 类文本的词语tf-idf权重------
-------这里输出第 1616 类文本的词语tf-idf权重------
-------这里输出第 1617 类文本的词语tf-idf权重------
-------这里输出第 1618 类文本的词语t

-------这里输出第 2331 类文本的词语tf-idf权重------
-------这里输出第 2332 类文本的词语tf-idf权重------
-------这里输出第 2333 类文本的词语tf-idf权重------
-------这里输出第 2334 类文本的词语tf-idf权重------
-------这里输出第 2335 类文本的词语tf-idf权重------
-------这里输出第 2336 类文本的词语tf-idf权重------
-------这里输出第 2337 类文本的词语tf-idf权重------
-------这里输出第 2338 类文本的词语tf-idf权重------
-------这里输出第 2339 类文本的词语tf-idf权重------
-------这里输出第 2340 类文本的词语tf-idf权重------
-------这里输出第 2341 类文本的词语tf-idf权重------
-------这里输出第 2342 类文本的词语tf-idf权重------
-------这里输出第 2343 类文本的词语tf-idf权重------
-------这里输出第 2344 类文本的词语tf-idf权重------
-------这里输出第 2345 类文本的词语tf-idf权重------
-------这里输出第 2346 类文本的词语tf-idf权重------
-------这里输出第 2347 类文本的词语tf-idf权重------
-------这里输出第 2348 类文本的词语tf-idf权重------
-------这里输出第 2349 类文本的词语tf-idf权重------
-------这里输出第 2350 类文本的词语tf-idf权重------
-------这里输出第 2351 类文本的词语tf-idf权重------
-------这里输出第 2352 类文本的词语tf-idf权重------
-------这里输出第 2353 类文本的词语tf-idf权重------
-------这里输出第 2354 类文本的词语tf-idf权重------
-------这里输出第 2355 类文本的词语tf-idf权重------
-------这里输出第 2356 类文本的词语t

-------这里输出第 3067 类文本的词语tf-idf权重------
-------这里输出第 3068 类文本的词语tf-idf权重------
-------这里输出第 3069 类文本的词语tf-idf权重------
-------这里输出第 3070 类文本的词语tf-idf权重------
-------这里输出第 3071 类文本的词语tf-idf权重------
-------这里输出第 3072 类文本的词语tf-idf权重------
-------这里输出第 3073 类文本的词语tf-idf权重------
-------这里输出第 3074 类文本的词语tf-idf权重------
-------这里输出第 3075 类文本的词语tf-idf权重------
-------这里输出第 3076 类文本的词语tf-idf权重------
-------这里输出第 3077 类文本的词语tf-idf权重------
-------这里输出第 3078 类文本的词语tf-idf权重------
-------这里输出第 3079 类文本的词语tf-idf权重------
-------这里输出第 3080 类文本的词语tf-idf权重------
-------这里输出第 3081 类文本的词语tf-idf权重------
-------这里输出第 3082 类文本的词语tf-idf权重------
-------这里输出第 3083 类文本的词语tf-idf权重------
-------这里输出第 3084 类文本的词语tf-idf权重------
-------这里输出第 3085 类文本的词语tf-idf权重------
-------这里输出第 3086 类文本的词语tf-idf权重------
-------这里输出第 3087 类文本的词语tf-idf权重------
-------这里输出第 3088 类文本的词语tf-idf权重------
-------这里输出第 3089 类文本的词语tf-idf权重------
-------这里输出第 3090 类文本的词语tf-idf权重------
-------这里输出第 3091 类文本的词语tf-idf权重------
-------这里输出第 3092 类文本的词语t

...

-------这里输出第 8497 类文本的词语tf-idf权重------
-------这里输出第 8498 类文本的词语tf-idf权重------
-------这里输出第 8499 类文本的词语tf-idf权重------
-------这里输出第 8500 类文本的词语tf-idf权重------
-------这里输出第 8501 类文本的词语tf-idf权重------
-------这里输出第 8502 类文本的词语tf-idf权重------
-------这里输出第 8503 类文本的词语tf-idf权重------
-------这里输出第 8504 类文本的词语tf-idf权重------
-------这里输出第 8505 类文本的词语tf-idf权重------
-------这里输出第 8506 类文本的词语tf-idf权重------
-------这里输出第 8507 类文本的词语tf-idf权重------
-------这里输出第 8508 类文本的词语tf-idf权重------
-------这里输出第 8509 类文本的词语tf-idf权重------
-------这里输出第 8510 类文本的词语tf-idf权重------
-------这里输出第 8511 类文本的词语tf-idf权重------
-------这里输出第 8512 类文本的词语tf-idf权重------
-------这里输出第 8513 类文本的词语tf-idf权重------
-------这里输出第 8514 类文本的词语tf-idf权重------
-------这里输出第 8515 类文本的词语tf-idf权重------
-------这里输出第 8516 类文本的词语tf-idf权重------
-------这里输出第 8517 类文本的词语tf-idf权重------
-------这里输出第 8518 类文本的词语tf-idf权重------
-------这里输出第 8519 类文本的词语tf-idf权重------
-------这里输出第 8520 类文本的词语tf-idf权重------
-------这里输出第 8521 类文本的词语tf-idf权重------
-------这里输出第 8522 类文本的词语t

-------这里输出第 9174 类文本的词语tf-idf权重------
-------这里输出第 9175 类文本的词语tf-idf权重------
-------这里输出第 9176 类文本的词语tf-idf权重------
-------这里输出第 9177 类文本的词语tf-idf权重------
-------这里输出第 9178 类文本的词语tf-idf权重------
-------这里输出第 9179 类文本的词语tf-idf权重------
-------这里输出第 9180 类文本的词语tf-idf权重------
-------这里输出第 9181 类文本的词语tf-idf权重------
-------这里输出第 9182 类文本的词语tf-idf权重------
-------这里输出第 9183 类文本的词语tf-idf权重------
-------这里输出第 9184 类文本的词语tf-idf权重------
-------这里输出第 9185 类文本的词语tf-idf权重------
-------这里输出第 9186 类文本的词语tf-idf权重------
-------这里输出第 9187 类文本的词语tf-idf权重------
-------这里输出第 9188 类文本的词语tf-idf权重------
-------这里输出第 9189 类文本的词语tf-idf权重------
-------这里输出第 9190 类文本的词语tf-idf权重------
-------这里输出第 9191 类文本的词语tf-idf权重------
-------这里输出第 9192 类文本的词语tf-idf权重------
-------这里输出第 9193 类文本的词语tf-idf权重------
-------这里输出第 9194 类文本的词语tf-idf权重------
-------这里输出第 9195 类文本的词语tf-idf权重------
-------这里输出第 9196 类文本的词语tf-idf权重------
-------这里输出第 9197 类文本的词语tf-idf权重------
-------这里输出第 9198 类文本的词语tf-idf权重------
-------这里输出第 9199 类文本的词语t

-------这里输出第 10162 类文本的词语tf-idf权重------
-------这里输出第 10163 类文本的词语tf-idf权重------
-------这里输出第 10164 类文本的词语tf-idf权重------
-------这里输出第 10165 类文本的词语tf-idf权重------
-------这里输出第 10166 类文本的词语tf-idf权重------
-------这里输出第 10167 类文本的词语tf-idf权重------
-------这里输出第 10168 类文本的词语tf-idf权重------
-------这里输出第 10169 类文本的词语tf-idf权重------
-------这里输出第 10170 类文本的词语tf-idf权重------
-------这里输出第 10171 类文本的词语tf-idf权重------
-------这里输出第 10172 类文本的词语tf-idf权重------
-------这里输出第 10173 类文本的词语tf-idf权重------
-------这里输出第 10174 类文本的词语tf-idf权重------
-------这里输出第 10175 类文本的词语tf-idf权重------
-------这里输出第 10176 类文本的词语tf-idf权重------
-------这里输出第 10177 类文本的词语tf-idf权重------
-------这里输出第 10178 类文本的词语tf-idf权重------
-------这里输出第 10179 类文本的词语tf-idf权重------
-------这里输出第 10180 类文本的词语tf-idf权重------
-------这里输出第 10181 类文本的词语tf-idf权重------
-------这里输出第 10182 类文本的词语tf-idf权重------
-------这里输出第 10183 类文本的词语tf-idf权重------
-------这里输出第 10184 类文本的词语tf-idf权重------
-------这里输出第 10185 类文本的词语tf-idf权重------
-------这里输出第 10186 类文本的词语tf-idf权重------


-------这里输出第 10760 类文本的词语tf-idf权重------
-------这里输出第 10761 类文本的词语tf-idf权重------
-------这里输出第 10762 类文本的词语tf-idf权重------
-------这里输出第 10763 类文本的词语tf-idf权重------
-------这里输出第 10764 类文本的词语tf-idf权重------
-------这里输出第 10765 类文本的词语tf-idf权重------
-------这里输出第 10766 类文本的词语tf-idf权重------
-------这里输出第 10767 类文本的词语tf-idf权重------
-------这里输出第 10768 类文本的词语tf-idf权重------
-------这里输出第 10769 类文本的词语tf-idf权重------
-------这里输出第 10770 类文本的词语tf-idf权重------
-------这里输出第 10771 类文本的词语tf-idf权重------
-------这里输出第 10772 类文本的词语tf-idf权重------
-------这里输出第 10773 类文本的词语tf-idf权重------
-------这里输出第 10774 类文本的词语tf-idf权重------
-------这里输出第 10775 类文本的词语tf-idf权重------
-------这里输出第 10776 类文本的词语tf-idf权重------
-------这里输出第 10777 类文本的词语tf-idf权重------
-------这里输出第 10778 类文本的词语tf-idf权重------
-------这里输出第 10779 类文本的词语tf-idf权重------
-------这里输出第 10780 类文本的词语tf-idf权重------
-------这里输出第 10781 类文本的词语tf-idf权重------
-------这里输出第 10782 类文本的词语tf-idf权重------
-------这里输出第 10783 类文本的词语tf-idf权重------
-------这里输出第 10784 类文本的词语tf-idf权重------


-------这里输出第 11408 类文本的词语tf-idf权重------
-------这里输出第 11409 类文本的词语tf-idf权重------
-------这里输出第 11410 类文本的词语tf-idf权重------
-------这里输出第 11411 类文本的词语tf-idf权重------
-------这里输出第 11412 类文本的词语tf-idf权重------
-------这里输出第 11413 类文本的词语tf-idf权重------
-------这里输出第 11414 类文本的词语tf-idf权重------
-------这里输出第 11415 类文本的词语tf-idf权重------
-------这里输出第 11416 类文本的词语tf-idf权重------
-------这里输出第 11417 类文本的词语tf-idf权重------
-------这里输出第 11418 类文本的词语tf-idf权重------
-------这里输出第 11419 类文本的词语tf-idf权重------
-------这里输出第 11420 类文本的词语tf-idf权重------
-------这里输出第 11421 类文本的词语tf-idf权重------
-------这里输出第 11422 类文本的词语tf-idf权重------
-------这里输出第 11423 类文本的词语tf-idf权重------
-------这里输出第 11424 类文本的词语tf-idf权重------
-------这里输出第 11425 类文本的词语tf-idf权重------
-------这里输出第 11426 类文本的词语tf-idf权重------
-------这里输出第 11427 类文本的词语tf-idf权重------
-------这里输出第 11428 类文本的词语tf-idf权重------
-------这里输出第 11429 类文本的词语tf-idf权重------
-------这里输出第 11430 类文本的词语tf-idf权重------
-------这里输出第 11431 类文本的词语tf-idf权重------
-------这里输出第 11432 类文本的词语tf-idf权重------


In [603]:


newDataDict={}
for i in range(len(tags.tagID)):
    newDataDict[tags.tagID[i]]=newData[i]

frequentItemDict={}
for i in frequentItem:
    temp=np.zeros(10)
    for j in list(user_taggedartists_timestamps[user_taggedartists_timestamps.artistID==i].tagID):
        temp+=newDataDict[j]
    frequentItemDict[i]=temp/np.linalg.norm(temp, ord=2)

clf1 = KMeans(n_clusters=10)
s1 = clf1.fit(list(frequentItemDict.values()))
#每个样本所属的簇
frequentItemCluster={}
for i in range(len(frequentItem)):
    frequentItemCluster[frequentItem[i]]=clf1.labels_[i]
frequentItemVec={}
for i in range(10):
    frequentItemVec[i]=np.zeros(10)
for i in frequentItem:
    frequentItemVec[frequentItemCluster[i]]+=frequentItemDict[i]
for i in range(10):
    frequentItemVec[i]=frequentItemVec[i]/np.linalg.norm(frequentItemVec[i], ord=2)

stage1Data=user_taggedartists_timestamps[:50000]
stage2Data=user_taggedartists_timestamps[50000:100000]
stage3Data=user_taggedartists_timestamps[100000:150000]
stage4Data=user_taggedartists_timestamps[150000:]
frequentUserDict1={}
frequentUserDict2={}
frequentUserDict3={}
frequentUserDict4={}
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage1Data[stage1Data.userID==i].tagID):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict1[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict1[i]=temp/np.linalg.norm(temp, ord=2)
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage2Data[stage2Data.userID==i].tagID):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict2[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict2[i]=temp/np.linalg.norm(temp, ord=2)
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage3Data[stage3Data.userID==i].tagID):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict3[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict3[i]=temp/np.linalg.norm(temp, ord=2)
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage4Data[stage4Data.userID==i].tagID):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict4[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict4[i]=temp/np.linalg.norm(temp, ord=2)

freqUserFeatureDict={}
for i in frequentUser:
    freqUserFeatureDict[i]=[]
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict1[i])
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict2[i])
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict3[i])
    for j in range(len(freqData)-3*len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict4[i])

In [651]:
def feedback(i,k,t):
    print(k,frequentItemCluster[trueResultFlow[t]])
    if k==frequentItemCluster[trueResultFlow[t]]:
        y=1
    else:
        y=0
    return y,1
def CB(alpha,x,M):
    return alpha*np.sqrt(np.dot(np.dot(x.T,np.linalg.inv(M)),x))
T=len(userFlow)
num_users=len(frequentUser)
fixedItemSet=frequentItemVec
K=10
d=10